## Import statements

In [7]:
import sys
sys.path.insert(0, '/Users/ahell/Documents/Python Projects/AIToolkit')

import os
from dotenv import load_dotenv, find_dotenv
from ai_toolkit import AIModel, PromptBuilder, FileReader, FileWriter, AIProcess, Function, APIRequest, AITool
from ai_toolkit.operations import ExtractKey, ConvertToJSON, Passthrough, Calculator
from IPython.display import display, Markdown, HTML, clear_output

In [8]:
# Load in the environment variables
load_dotenv(find_dotenv())

True

In [9]:
AIModel.ALL_MODEL_NAMES

['gpt-3.5-turbo',
 'gpt-4',
 'gpt-3.5-turbo-16k',
 'claude-1',
 'claude-1-100k',
 'claude-instant-1',
 'claude-instant-1-100k',
 'respell-gpt-4-wrapper']

In [10]:
client_survey_id = 'adf4f406-757c-4e7d-b881-add3de4bbaf4'
client_name = 'Marcfirst'

## Dimension Analysis

In [11]:
request_url = f"https://dev-api.insite.ledgestone.com/api/results/comments/{client_survey_id}/open_ended"
request_method = "GET"
request_headers = {
    "Authorization": f"Bearer {os.environ['BEARER_TOKEN']}",
    "Content-Type": "application/json"
}
comments_request = APIRequest("Comments Request").set_input(
    url=request_url, method=request_method, headers=request_headers)
comments_request.process()

Comments Request (APIRequest)

In [12]:
comments_function = ExtractKey("Comments List Extractor").set_input(
    input=comments_request, key_name="comments")
comments_function.process()
comments_list = comments_function.get_output()
print(comments_list)

[{'comment': 'I am happy to have the opportunity to work at Marcfirst. ', 'nps_category': 'promoter', 'comment_number': 1, 'prompt_response_id': '019a6a43-09d2-4452-ac87-bb569e91f5dc'}, {'comment': 'Seems like this is started to be a corporation instead of a non-profit.', 'nps_category': 'detractor', 'comment_number': 2, 'prompt_response_id': '04f38390-8d79-427e-933e-4cfd41ad9a1c'}, {'comment': 'Marcfirst has been wonderful, and I look forward to growing with as we continue expanding and helping as many individuals as possible.', 'nps_category': 'promoter', 'comment_number': 3, 'prompt_response_id': '1ca2239e-359e-429c-b226-fc9845a94d00'}, {'comment': 'i think this company totally fails to recognize the accomplishments and acheivements of personnel that worked hard to get them to where they are today. They see workers that have had a long tenure with them as expendable and often frind a way to let them go, expecially if they are not enthusiastic with new ways of doing things or changes

In [13]:
# from typing import Any

# class DimensionAnalyzerOutputFunction(CustomCodeBlock):
#     def code(self):
#         analysis = self.get_input("analysis")

# extract_comment = AITool().set_input(object="Function"...)
# dimension_prompt = AITool().set_input(object="PromptBuilder"...)

# extract_comment = ExtractComment("Name").set_input(name_of_input="comments_list", key_name="comment")
# dimension_analyzer_json = ConvertToJSON("Name").set_input(name_of_input="dimension_analyzer_llm")

comment_dict_input = Passthrough("Comment Dict Input").set_input(
    name_of_input="comment_dict")

extract_comment = ExtractKey("Extract Comment").set_input(
    input=comment_dict_input, key_name="comment")

extract_comment_num = ExtractKey("Extract Comment Number").set_input(
    input=comment_dict_input, key_name="comment_number")

extract_prompt_response_id = ExtractKey("Extract Prompt Response ID").set_input(
    input=comment_dict_input, key_name="prompt_response_id")

dimension_prompt_file = FileReader("Dimension Analyzer File").set_input(
    file_path="dimension_prompt.txt")

dimension_prompt = PromptBuilder("Dimension Analyzer Prompt").set_input(
    template=dimension_prompt_file, comment=extract_comment)

dimension_analyzer_llm = AIModel("Dimension Analyzer LLM").set_input(
    model_name="respell-gpt-4-wrapper", prompt=dimension_prompt)

dimension_analyzer_json = ConvertToJSON("Dimension Analyzer JSON").set_input(
    input=dimension_analyzer_llm)

convert_support_and_regognition_to_value_function = """
    analysis = get_input("analysis")
    if 'Support & Recognition' in analysis:
        analysis['Value'] = analysis.pop('Support & Recognition')
    return analysis
"""

dimension_analyzer_output_function = """
    comment_number = get_input("comment_number")
    comment = get_input("comment")
    prompt_response_id = get_input("prompt_response_id")
    analysis = get_input("analysis")
    output = list()
    for dimension, anal in analysis.items():
        output.append({
            "comment_number": comment_number,
            "comment": comment,
            "prompt_response_id": prompt_response_id,
            "dimension": dimension,
            "analysis": anal["analysis"],
            "sentiment": anal["sentiment"]
            })

    return output
"""

convert_support_and_recognition_to_value = Function("Convert Support & Recognition to Value").set_input(
    function=convert_support_and_regognition_to_value_function, analysis=dimension_analyzer_json)

dimension_analyzer_output = Function("Dimension Analyzer Output").set_input(
    function=dimension_analyzer_output_function, 
    comment_number=extract_comment_num, 
    comment=extract_comment,
    prompt_response_id=extract_prompt_response_id, 
    analysis=convert_support_and_recognition_to_value)

dimension_analyzer = AIProcess("Dimension Analyzer")
dimension_analyzer.expose_input("comment_dict", comment_dict_input)
dimension_analyzer.expose_output(dimension_analyzer_output)

Dimension Analyzer (AIProcess)

In [14]:
output = list()
for i in range(len(comments_list)):
    clear_output(wait=True)
    print(f"Processing comment {i+1} of {len(comments_list)}")
    dimension_analyzer.set_input(comment_dict=comments_list[i])
    dimension_analyzer.process()
    output.extend(dimension_analyzer.get_output())

Processing comment 36 of 36
Processing Comment Dict Input (Passthrough)
Processing Extract Comment (ExtractKey)
Processing Extract Comment Number (ExtractKey)
Processing Extract Prompt Response ID (ExtractKey)
Processing Dimension Analyzer File (FileReader)
Processing Dimension Analyzer Prompt (PromptBuilder)
Processing Dimension Analyzer LLM (AIModel)
Processing Dimension Analyzer JSON (ConvertToJSON)
Processing Convert Support & Recognition to Value (Function)
Processing Dimension Analyzer Output (Function)


In [15]:
dimension_results_json = FileWriter("Save Dimension Analysis JSON").set_input(file_path=f"Clients/{client_name}/dimensions_json.json", data=output)
dimension_results_json.process()
dimension_results_json.get_output()

'File successfully written'

## Driver Analysis

In [26]:
dimensions_to_analyze = [
    "Accomplishment",
    "Alignment",
    "Compensation & Benefits",
    "Connection",
    "Diversity",
    "Purpose",
    "Security",
    "Value",
    "Work Environment",
    "Workload & Stress"
]
print(len(output))
dimension_ouput_for_driver_analysis = list()
for comment_analysis in output:
    dimension = comment_analysis["dimension"]
    if dimension in dimensions_to_analyze:
        dimension_ouput_for_driver_analysis.append(comment_analysis)
    else:
        print(f"Skipping {dimension}")

print(len(dimension_ouput_for_driver_analysis))

81
Skipping Not Applicable
Skipping Not Applicable
Skipping Not Applicable
78


In [34]:
comment_dict_input = Passthrough("Comment Dict Input").set_input(
    name_of_input="comment_dict")

extract_comment = ExtractKey("Extract Comment").set_input(
    input=comment_dict_input, key_name="comment")

extract_comment_num = ExtractKey("Extract Comment Number").set_input(
    input=comment_dict_input, key_name="comment_number")

extract_prompt_response_id = ExtractKey("Extract Prompt Response ID").set_input(
    input=comment_dict_input, key_name="prompt_response_id")

extract_dimension = ExtractKey("Extract Dimension").set_input(
    input=comment_dict_input, key_name="dimension")

extract_analysis = ExtractKey("Extract Analysis").set_input(
    input=comment_dict_input, key_name="analysis")

extract_sentiment = ExtractKey("Extract Sentiment").set_input(
    input=comment_dict_input, key_name="sentiment")

driver_prompt_file_name_function = """
    dimension = get_input("dimension")
    return f"driver_prompt_{dimension.lower().replace(' ', '_').replace('&', 'and')}.txt"
"""

driver_prompt_file_name = Function("Driver Prompt File Name").set_input(
    function=driver_prompt_file_name_function, dimension=extract_dimension)

driver_prompt_file = FileReader("Driver Analyzer File").set_input(
    file_path=driver_prompt_file_name)

driver_prompt = PromptBuilder("Driver Analyzer Prompt").set_input(
    template=driver_prompt_file, comment=extract_comment, value_analysis=extract_analysis)

driver_analyzer_llm = AIModel("Driver Analyzer LLM").set_input(
    model_name="gpt-4", prompt=driver_prompt)

driver_analyzer_json = ConvertToJSON("Driver Analyzer JSON").set_input(
    input=driver_analyzer_llm)

driver_analyzer_output_function = """
    comment_num = get_input("comment_number")
    comment = get_input("comment")
    prompt_response_id = get_input("prompt_response_id")
    dimension = get_input("dimension")
    analysis = get_input("analysis")
    sentiment = get_input("sentiment")
    takeaway = get_input("takeaway")
    output = list()
    for driver, takeaway in takeaway.items():
        output.append({
            'prompt_response_id': prompt_response_id,
            'comment_num': comment_num,
            'dimension': dimension,
            'sentiment': sentiment,
            'comment': comment,
            'analysis': analysis,
            'driver': driver,
            'takeaway': takeaway
        })

    return output
"""

driver_analyzer_output = Function("Driver Analyzer Output").set_input(
    function=driver_analyzer_output_function, 
    comment_number=extract_comment_num,
    comment=extract_comment,
    prompt_response_id=extract_prompt_response_id,
    dimension=extract_dimension,
    analysis=extract_analysis,
    sentiment=extract_sentiment,
    takeaway=driver_analyzer_json)

driver_analyzer = AIProcess("Driver Analyzer")
driver_analyzer.expose_input("comment_dict", comment_dict_input)
driver_analyzer.expose_output(driver_analyzer_output)

Driver Analyzer (AIProcess)

In [35]:
driver_output = list()
for i in range(len(dimension_ouput_for_driver_analysis)):
    clear_output(wait=True)
    print(f"Processing comment {i+1} of {len(dimension_ouput_for_driver_analysis)}")
    driver_analyzer.set_input(comment_dict=dimension_ouput_for_driver_analysis[i])
    driver_analyzer.process()
    driver_output.extend(driver_analyzer.get_output())

Processing comment 12 of 78
Processing Comment Dict Input (Passthrough)
Processing Extract Comment (ExtractKey)
Processing Extract Comment Number (ExtractKey)
Processing Extract Prompt Response ID (ExtractKey)
Processing Extract Dimension (ExtractKey)
Processing Driver Prompt File Name (Function)
Processing Driver Analyzer File (FileReader)
Processing Extract Analysis (ExtractKey)
Processing Extract Sentiment (ExtractKey)
Processing Driver Analyzer Prompt (PromptBuilder)
Processing Driver Analyzer LLM (AIModel)


RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-1ra0eeBjtT69TLEagEMEAA9n on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.

In [32]:
import json
display(driver_analyzer_llm.get_output())

data = '{"Pride": "From this employee\'s experience, the uncertainty and fear of job security is negatively affecting their pride in the organization; therefore, leaders need to address these concerns transparently and promptly because such uncertainties can divert focus from the core purpose of serving families."}'
json.loads(data)

'"Pride": "From this employee\'s experience, the uncertainty and fear of job security is negatively affecting their pride in the organization; therefore, leaders need to address these concerns transparently and promptly because such uncertainties can divert focus from the core purpose of serving families."'

{'Pride': "From this employee's experience, the uncertainty and fear of job security is negatively affecting their pride in the organization; therefore, leaders need to address these concerns transparently and promptly because such uncertainties can divert focus from the core purpose of serving families."}

In [ ]:
driver_results_json = FileWriter("Save Driver Analysis JSON").set_input(file_path=f"Clients/{client_name}/drivers_json.json", data=driver_output)
driver_results_json.process()
driver_results_json.get_output()

In [ ]:
keep_keys = ['Meaning', 'Pride', 'Significance', 'Supervisor', 'Organization', 'Appreciation', 'Team', 'Empowerment', 'Investment', 'Impact', 'Clarity', 'Advancement', 'Strategy', 'Vision', 'Mastery', 'Trust', 'Equity', 'Execution']
extra_keep_keys = ['Compensation', 'Work Environment', 'Workload & Stress', 'Diversity']

result_dict = {}
for anal in driver_output:
        if not (anal['driver'] in keep_keys or anal['driver'] in extra_keep_keys):
            continue
        dimension = row['Comment']['dimension']
        comment = row['Comment']['comment']
        analysis = row['Comment']['analysis']
        comment_num = row['Comment']['comment_number']
        takeaway = row['Takeaway']
        for key, value in takeaway.items():
            if key in result_dict:
                result_dict[key].append(f'{comment_num} - {value}')
            else:
                result_dict[key] = [f'{comment_num} - {value}']